# Introduction

Here is the jupyter notebook for those who have signed up for the [competition](https://codalab.lisn.upsaclay.fr/competitions/8440?secret_key=51d5952f-d68d-47d9-baef-6032445dea01) in NTU.

## Goal
The competition's goal is to maximize the accumulative return rate of the test input. Based on the current price information, your agent should generate a portfolio weight which will give the hightest return rate defined below. 

## Dataset
In this competition, you will be given 1 complete datasets as training dataset and 2 incomplete datasets as test datasets, where there is no feature `close`.

In both complete and incomplete datasets, there are 15 assets.
### Data Visualization
Here is a glance of how the train data look like

In [44]:
import numpy as np 
import pandas as pd
train=pd.read_csv("data/train.csv",index_col=0)
train.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,date,close,tic
0,-0.008320,0.002999,-0.010062,-0.143726,0.005374,-0.010348,-0.011415,0.002751,-0.061124,-0.072128,-0.126481,0,14.765714,0
0,0.005332,0.005803,-0.005646,-0.225326,-0.005459,0.007794,-0.008521,-0.008222,-0.017393,-0.031253,-0.042012,0,63.759998,1
0,-0.004561,0.002902,-0.017413,-0.136140,-0.003100,0.004561,-0.004885,-0.017185,-0.029824,-0.045096,-0.052918,0,48.240002,2
0,-0.003094,0.003632,-0.009956,-0.181742,0.001482,0.002922,-0.007421,-0.004776,-0.002812,-0.013054,-0.001687,0,74.330002,3
0,-0.015182,0.006958,-0.015182,-0.252029,0.009257,0.010322,-0.004803,-0.010627,-0.001714,-0.005438,0.006133,0,94.849998,4


In the RL setting, we view all 15 stocks's feature information as state, which is the input of your model.

Here is a glance of what the state in timestamp 0 looks like

In [45]:
# technical_indicator_list=["feature_0","feature_1","feature_2","feature_3","feature_4","feature_5","feature_6","feature_7","feature_8","feature_9","feature_10"]
# train.loc[0][technical_indicator_list]

### Reward Calculation
Based on the state in timestamp 0, we use random portfolio weights to demonstrate how the reward of the random action is calculated.

In [46]:
# generate random action
random_score=np.random.rand(15)
random_weights=np.exp(random_score)/np.sum(np.exp(random_score))
# calculate return_rate for every single ticker of day 0
close_price_0=train.loc[0].close.values
close_price_1=train.loc[1].close.values
single_return_rate=close_price_1/close_price_0-1
# calculate the return rate of the random action
return_rate_action=np.sum(random_weights*single_return_rate)
return_rate_action

0.0037674239218332445

We provide a [Trading Environment](https://github.com/TradeMaster-NTU/TradeMaster/blob/main/env/PM/portfolio_for_EIIE.py) to automate the process of state stepping and reward calculation.

## Baseline Implementation
Here, we provide a baseline for the competition using one of the algorithms in TradeMaster. You can follow the rest jupyter notebook to create a borderline submission.


# Installation

Here we provide the [video tutorial](https://www.youtube.com/watch?v=7rtqFT9I4uo&t=12s) for you to install this project to better participate in the competition.

# RL enviornment & baselines

### Running Environment Set Up
First, we need to add the project to our system path because the setting of jupter notebook is a little different from that of py document.
Then, we need to load the document we need to import the module we need

In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
import sys
sys.path.append("..")
from agent.EIIE.model import EIIE_con, EIIE_lstm, EIIE_rnn, EIIE_critirc
import argparse
from agent.EIIE.util import *
from env.PM.portfolio_for_EIIE import Tradingenv
from logging import raiseExceptions
from stat import S_ENFMT
import torch.nn as nn
import pandas as pd
import sys
from agent.EIIE.trader import trader


### Config and Hyperparameters Adjustment


Below is a part where you can adjust the default hyperparameters
More specifically, besides the hyperparameters the DPG need, it also contains information like the path of the config file of the training and validing environment. 

In [49]:
parser = argparse.ArgumentParser()
parser.add_argument("--random_seed",
                    type=int,
                    default=12345,
                    help="random seed number")
parser.add_argument(
    "--env_config_path",
    type=str,
    default="config/input_config/env/portfolio/portfolio_for_EIIE/",
    help="the path for storing the downloaded data")
parser.add_argument(
    "--net_type",
    choices=["conv", "lstm", "rnn"],
    default="rnn",
    help="the name of the model",
)
parser.add_argument(
    "--num_hidden_nodes",
    type=int,
    default=32,
    help="the number of hidden nodes in lstm or rnn",
)
parser.add_argument(
    "--num_out_channel",
    type=int,
    default=2,
    help="the number of channel",
)
parser.add_argument(
    "--gamma",
    type=float,
    default=0.99,
    help="the gamma for DPG",
)
parser.add_argument(
    "--model_path",
    type=str,
    default="result/EIIE/trained_model",
    help="the path for trained model",
)
parser.add_argument(
    "--result_path",
    type=str,
    default="result/EIIE/test_result",
    help="the path for test result",
)
parser.add_argument(
    "--num_epoch",
    type=int,
    default=1,
    help="the number of epoch we train",
)


_StoreAction(option_strings=['--num_epoch'], dest='num_epoch', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, help='the number of epoch we train', metavar=None)

### Agent Building & Training

In [50]:
args = parser.parse_args(args=[])
agent=trader(args)
# agent.train_with_valid()

print(agent.train_env_config)


{'df_path': 'data/train.csv', 'initial_amount': 100000, 'length_day': 3, 'tech_indicator_list': ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10'], 'transaction_cost_pct': 0.0}


### Testing & Generate Submission Files

In [51]:
# test_information_1=pd.read_csv("data/test_input_1.csv",index_col=0)
# test_information_2=pd.read_csv("data/test_input_2.csv",index_col=0)
# technical_indicator=["feature_0","feature_1","feature_2","feature_3","feature_4","feature_5","feature_6","feature_7","feature_8","feature_9","feature_10"]

In [52]:
# action_list_1=[]
# for date in test_information_1.index.unique():
#     s=test_information_1[test_information_1.index==date][technical_indicator].values
#     shape=s.shape
#     s=s.reshape(shape[0],1,shape[1])
#     s=torch.from_numpy(s).float()
#     action=agent.net(s)
#     #here the origional action for the environment consider the cash, which is more pracitical in real world, but for 
#     #the competition, we only need the last 15 weights for the assets, therefore we need to normalize the result as well
#     action=action.detach().float().numpy()
#     action=action[1:]/np.sum(action[1:])
#     action_list_1.append(action)
# action_list_1=np.array(action_list_1)
# action_list_1=action_list_1.astype(float)
# np.save("result/EIIE/action/action1.npy",action_list_1)

In [53]:
# action_list_2=[]
# for date in test_information_2.index.unique():
#     s=test_information_2[test_information_2.index==date][technical_indicator].values
#     shape=s.shape
#     s=s.reshape(shape[0],1,shape[1])
#     s=torch.from_numpy(s).float()
#     action=agent.net(s)
#     #here the origional action for the environment consider the cash, which is more pracitical in real world, but for 
#     #the competition, we only need the last 15 weights for the assets, therefore we need to normalize the result as well
#     action=action.detach().float().numpy()
#     action=action[1:]/np.sum(action[1:])
#     action_list_2.append(action)
# action_list_2=np.array(action_list_2)
# action_list_2=action_list_2.astype(float)
# np.save("result/EIIE/action/action2.npy",action_list_2)

Next, Compress the 2 file as a zip file as submission file.
Please notice that the zip file should only contain this 2 files name after `action1.npy` and `action2.npy` with no external folder. 


The next is core code of the agent.
## RL Environment Configuration

In TradeMaster, you need to provide 3 RL environment configuration before building an agent. The TradeMaster agents will automatically follow the process using the training RL environment to train the agent, test it in the validation environment and pick the best model of training and pose it in the testing environment.

In [54]:
train_env_instance=agent.train_env_instance
valid_env_instance=agent.valid_env_instance
test_env_instance=agent.train_env_instance

In [55]:
# Reseting the environment, means to clear all the history and start from the begining. 
# It will return the initial state  
# Here is an example of posing the random action to the train_env_instance
# s=train_env_instance.reset()
# action=np.random.rand(16)
# done=False
# while not done:
#     old_state = s
#     s, reward, done, _ =train_env_instance.step(action)

In [56]:
def check_action_score(action_score_list):
    true_action_list = []
    for action_score in action_score_list:
        if np.sum(action_score) == 1:
            action = action_score
        else:
            action = np.exp(action_score) / np.sum(np.exp(action_score))
        true_action_list.append(action)
    return np.array(true_action_list)

next is the core code of EIIE, I will decompose it so that you can understand the process of training, making it easier for you to build your own agent

In [57]:
# define the net
# Just like the supervised leanrning process, we need a net to regress something, here the EIIE is Actor-Critic RL model where we need a actor to to generate policy and a critic\
# to judge whether the state is good or not

import torch
from torch import nn
import numpy as np


class EIIE_con(torch.nn.Module):
    def __init__(self, in_channels, out_channels, length, kernel_size=1):
        super(EIIE_con, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.length = length
        self.act = torch.nn.ReLU(inplace=False)
        self.con1d = nn.Conv1d(self.in_channels,
                               self.out_channels,
                               kernel_size=1)
        self.con2d = nn.Conv1d(self.out_channels,
                               1,
                               kernel_size=1)
        self.con3d = nn.Conv1d(1, 1, kernel_size=1)
        self.para = torch.nn.Parameter(torch.ones(1).requires_grad_())

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.con1d(x)
        x = self.act(x)
        x = self.con2d(x)
        x = self.act(x)
        x = self.con3d(x)
        x = x.view(-1)

        # self.linear2 = nn.Linear(len(x), len(x) + 1)
        # x = self.linear2(x)
        x = torch.cat((x, self.para), dim=0)
        x = torch.softmax(x, dim=0)

        return x


class EIIE_lstm(nn.Module):
    def __init__(self, n_features, layer_num, n_hidden):
        super(EIIE_lstm, self).__init__()
        self.n_features = n_features
        self.n_hidden = n_hidden
        self.n_layers = layer_num
        self.lstm = nn.LSTM(input_size=n_features,
                            hidden_size=self.n_hidden,
                            num_layers=self.n_layers,
                            batch_first=True)
        self.linear = nn.Linear(self.n_hidden, 1)
        self.con3d = nn.Conv1d(1, 1, kernel_size=1)
        self.para = torch.nn.Parameter(torch.ones(1).requires_grad_())

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # print('1',lstm_out[:, -1, :].shape)
        # print('2',lstm_out[:, :, :].shape)
        # x = self.linear(lstm_out).transpose(1, 2)
        x = self.linear(lstm_out[:, -1, :]).view(-1, 1, 1)
        # print('3',x.transpose(1, 2).shape)        
        # print('4',x.shape)
        # print('5',x)
        x = self.con3d(x)
        # print('6',x.shape)
        x = x.view(-1)
        # print('7',x.shape)
        x = torch.cat((x, self.para), dim=0)
        x = torch.softmax(x, dim=0)
        # print(x.shape)
        return x


class EIIE_rnn(nn.Module):
    def __init__(self, n_features, layer_num, n_hidden):
        super(EIIE_rnn, self).__init__()
        self.n_features = n_features
        self.n_hidden = n_hidden
        self.n_layers = layer_num
        self.rnn = nn.RNN(input_size=n_features,
                          hidden_size=self.n_hidden,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.linear = nn.Linear(self.n_hidden, 1)
        self.con3d = nn.Conv1d(1, 1, kernel_size=1)
        self.para = torch.nn.Parameter(torch.ones(1).requires_grad_())

    def forward(self, x):
        lstm_out, _ = self.rnn(x)
        x = self.linear(lstm_out[:, -1, :]).view(-1, 1, 1)
        x = self.con3d(x)
        x = x.view(-1)
        x = torch.cat((x, self.para), dim=0)
        x = torch.softmax(x, dim=0)
        return x
    
class EIIE_critirc(nn.Module):
    def __init__(self, n_features, layer_num, n_hidden):
        super(EIIE_critirc, self).__init__()
        self.n_features = n_features
        self.n_hidden = n_hidden
        self.n_layers = layer_num
        self.lstm = nn.LSTM(input_size=n_features,
                            hidden_size=self.n_hidden,
                            num_layers=self.n_layers,
                            batch_first=True)
        self.linear = nn.Linear(self.n_hidden, 1)
        self.con3d = nn.Conv1d(1, 1, kernel_size=1)
        self.para = torch.nn.Parameter(torch.ones(1).requires_grad_())

    def forward(self, x, a):
        lstm_out, _ = self.lstm(x)
        x = self.linear(lstm_out[:, -1, :]).view(-1, 1, 1)
        x = self.con3d(x)
        x = x.view(-1)
        x = torch.cat((x, self.para, a), dim=0)
        x = torch.nn.ReLU(inplace=False)(x)
        number_nodes = len(x)
        self.linear2 = nn.Linear(number_nodes, 1)
        x = self.linear2(x)
        return x

In [58]:
# here are the code for the trader, the key lies on the learn function
model_name = "lstm_win3_noStandardized_2layer_16hidden_0.99g_epoch20"
result_folder_location = "result/"+ model_name + "/test_result/"
model_folder_location = "result/"+ model_name + "/trained_model"
class trader:
    def __init__(self):
        self.num_epoch = 20
        self.GPU_IN_USE = torch.cuda.is_available()
        self.device = torch.device('cpu' if self.GPU_IN_USE else 'cpu')
        self.model_path = model_folder_location
        if not os.path.exists(self.model_path):
            os.makedirs(self.model_path)
        self.result_path = result_folder_location
        if not os.path.exists(self.result_path):
            os.makedirs(self.result_path)
        self.train_env_instance = train_env_instance
        self.valid_env_instance = valid_env_instance
        self.test_env_instance = test_env_instance
        self.n_hidden = 16
        self.input_channel = 11
        self.layer_num = 2
        self.net = EIIE_lstm(self.input_channel, self.layer_num,
                           self.n_hidden)
        
        self.critic = EIIE_critirc(self.input_channel, self.layer_num,
                                  self.n_hidden)
        self.test_action_memory = []  # to store the
        self.optimizer_actor = torch.optim.Adam(self.net.parameters(), lr=1e-4)
        self.optimizer_critic = torch.optim.Adam(self.critic.parameters(),
                                                 lr=1e-4)
        self.memory_counter = 0
        self.memory_capacity = 1000
        self.s_memory = []
        self.a_memory = []
        self.r_memory = []
        self.sn_memory = []
        self.policy_update_frequency = 500
        self.critic_learn_time = 0
        self.gamma = 0.99
        self.mse_loss = nn.MSELoss()
        self.net = self.net.to(self.device)
        self.critic = self.critic.to(self.device)

    def store_transition(
        self,
        s,
        a,
        r,
        s_,
    ):  # 定义记忆存储函数 (这里输入为一个transition)

        self.memory_counter = self.memory_counter + 1
        if self.memory_counter < self.memory_capacity:
            self.s_memory.append(s)
            self.a_memory.append(a)
            self.r_memory.append(r)
            self.sn_memory.append(s_)
        else:
            number = self.memory_counter % self.memory_capacity
            self.s_memory[number - 1] = s
            self.a_memory[number - 1] = a
            self.r_memory[number - 1] = r
            self.sn_memory[number - 1] = s_

    def compute_single_action(self, state):
        state = torch.from_numpy(state).float().to(self.device)
        action = self.net(state)
        action = action.detach().cpu().numpy()
        return action

    def learn(self):
        # here is the core of the trader, it shows how the updates coming out
        # we first need to have some stored the transcation(s,a,r,s_) 
        length = len(self.s_memory)
        out1 = random.sample(range(length), int(length / 10))
        # random sample
        s_learn = []
        a_learn = []
        r_learn = []
        sn_learn = []
        for number in out1:
            s_learn.append(self.s_memory[number])
            a_learn.append(self.a_memory[number])
            r_learn.append(self.r_memory[number])
            sn_learn.append(self.sn_memory[number])
        self.critic_learn_time = self.critic_learn_time + 1
        # for the transcation we have stored, we need to update the actor and critic
        # for the actor, we need to comput the action and use the critic to judge the action
        # we need to update the actor so that for every action it choose, it can gain more scores from a critic than other action 
        # for the critic , we simply use the td_error to update it because it is MDP

        for bs, ba, br, bs_ in zip(s_learn, a_learn, r_learn, sn_learn):
            #update actor
            a = self.net(bs)
            q = self.critic(bs, a)
            a_loss = -torch.mean(q)
            self.optimizer_actor.zero_grad()
            a_loss.backward(retain_graph=True)
            self.optimizer_actor.step()
            #update critic
            a_ = self.net(bs_)
            q_ = self.critic(bs_, a_.detach())
            q_target = br + self.gamma * q_
            q_eval = self.critic(bs, ba.detach())
            # print(q_eval)
            # print(q_target)
            td_error = self.mse_loss(q_target.detach(), q_eval)
            # print(td_error)
            self.optimizer_critic.zero_grad()
            td_error.backward()
            self.optimizer_critic.step()

    def train_with_valid(self):
        # print(self.train_env_instance.length_day)
        rewards_list = []
        for i in range(self.num_epoch):
            j = 0
            done = False
            s = self.train_env_instance.reset()
            while not done:

                old_state = s
                # print('checking input size: ', torch.from_numpy(s).float().size())
                action = self.net(torch.from_numpy(s).float())
                s, reward, done, _ = self.train_env_instance.step(
                    action.detach().numpy())
                self.store_transition(
                    torch.from_numpy(old_state).float().to(self.device),
                    action,
                    torch.tensor(reward).float().to(self.device),
                    torch.from_numpy(s).float().to(self.device))
                j = j + 1
                if j % 200 == 1:
                    print(f"epoch {i} step {j}:")
                    print(f"action: {action}")
                    print(f"reward: {reward}")
                    self.learn()
            all_model_path = self.model_path + "/all_model/"
            best_model_path = self.model_path + "/best_model/"
            if not os.path.exists(all_model_path):
                os.makedirs(all_model_path)
            if not os.path.exists(best_model_path):
                os.makedirs(best_model_path)
            torch.save(self.net,
                       all_model_path + "actor_num_epoch_{}.pth".format(i))
            torch.save(self.critic,
                       all_model_path + "critic_num_epoch_{}.pth".format(i))
            s = self.valid_env_instance.reset()
            done = False
            rewards = 0
            while not done:

                old_state = s
                action = self.net(torch.from_numpy(s).float())
                s, reward, done, _ = self.valid_env_instance.step(
                    action.detach().numpy())
                rewards = rewards + reward
            rewards_list.append(rewards)
        max_reward = np.max(rewards_list)
        index = rewards_list.index(max_reward)
        actor_model_path = all_model_path + "actor_num_epoch_{}.pth".format(
            index)
        critic_model_path = all_model_path + "critic_num_epoch_{}.pth".format(
            index)
        self.net = torch.load(actor_model_path)
        self.critic = torch.load(critic_model_path)
        os.makedirs(best_model_path + str(max_reward))
        torch.save(self.net, best_model_path + "actor.pth")
        torch.save(self.critic, best_model_path + "critic.pth")

    def test(self):
        s = self.test_env_instance.reset()
        done = False
        while not done:
            old_state = s
            action = self.net(torch.from_numpy(s).float())
            s, reward, done, _ = self.test_env_instance.step(
                action.detach().numpy())
        df_return = self.test_env_instance.save_portfolio_return_memory()
        df_assets = self.test_env_instance.save_asset_memory()
        assets = df_assets["total assets"].values
        daily_return = df_return.daily_return.values
        df = pd.DataFrame()
        df["daily_return"] = daily_return
        df["total assets"] = assets
        if not os.path.exists(self.result_path):
            os.makedirs(self.result_path)
        df.to_csv(self.result_path + "/result.csv")

In [59]:
agent=trader()
agent.train_with_valid()
test_information_1=pd.read_csv("data/test_input_1.csv",index_col=0)
test_information_2=pd.read_csv("data/test_input_2.csv",index_col=0)
technical_indicator=["feature_0","feature_1","feature_2","feature_3","feature_4","feature_5","feature_6","feature_7","feature_8","feature_9","feature_10"]
action_list_1=[]
for date in test_information_1.index.unique():
    s=test_information_1[test_information_1.index==date][technical_indicator].values
    shape=s.shape
    s=s.reshape(shape[0],1,shape[1])
    s=torch.from_numpy(s).float()
    action=agent.net(s)
    #here the origional action for the environment consider the cash, which is more pracitical in real world, but for 
    #the competition, we only need the last 15 weights for the assets, therefore we need to normalize the result as well
    action=action.detach().float().numpy()
    action=action[1:]/np.sum(action[1:])
    action_list_1.append(action)
action_list_1=np.array(action_list_1)
action_list_1=action_list_1.astype(float)
# action_list_1 = check_action_score(action_list_1)
np.save(result_folder_location + "action1.npy",action_list_1)
action_list_2=[]
for date in test_information_2.index.unique():
    s=test_information_2[test_information_2.index==date][technical_indicator].values
    shape=s.shape
    s=s.reshape(shape[0],1,shape[1])
    s=torch.from_numpy(s).float()
    action=agent.net(s)
    #here the origional action for the environment consider the cash, which is more pracitical in real world, but for 
    #the competition, we only need the last 15 weights for the assets, therefore we need to normalize the result as well
    action=action.detach().float().numpy()
    action=action[1:]/np.sum(action[1:])
    action_list_2.append(action)
action_list_2=np.array(action_list_2)
action_list_2=action_list_2.astype(float)
# action_list_2 = check_action_score(action_list_2)
np.save(result_folder_location + "action2.npy",action_list_2)

epoch 0 step 1:
action: tensor([0.0521, 0.0521, 0.0521, 0.0521, 0.0521, 0.0521, 0.0521, 0.0522, 0.0521,
        0.0521, 0.0521, 0.0521, 0.0521, 0.0522, 0.0521, 0.2182],
       grad_fn=<SoftmaxBackward0>)
reward: 0.0029372061417642925
epoch 0 step 201:
action: tensor([0.0521, 0.0521, 0.0521, 0.0522, 0.0521, 0.0521, 0.0521, 0.0521, 0.0521,
        0.0521, 0.0521, 0.0521, 0.0521, 0.0521, 0.0522, 0.2183],
       grad_fn=<SoftmaxBackward0>)
reward: -0.013245675343268815
epoch 0 step 401:
action: tensor([0.0522, 0.0522, 0.0521, 0.0521, 0.0521, 0.0522, 0.0521, 0.0521, 0.0521,
        0.0521, 0.0521, 0.0521, 0.0521, 0.0521, 0.0521, 0.2180],
       grad_fn=<SoftmaxBackward0>)
reward: 0.006781093010880923
epoch 0 step 601:
action: tensor([0.0522, 0.0522, 0.0522, 0.0522, 0.0522, 0.0522, 0.0522, 0.0522, 0.0522,
        0.0522, 0.0522, 0.0522, 0.0522, 0.0522, 0.0522, 0.2170],
       grad_fn=<SoftmaxBackward0>)
reward: 0.00010860242879218163
epoch 0 step 801:
action: tensor([0.0522, 0.0522, 0.0522, 

Next, Compress the 2 file as a zip file as submission file.

# Playground

In [2]:
given = np.load("result/EIIE/action/action1.npy")
givenzip = np.load("result/EIIE/action/result_submission/action1.npy")
# ran = np.load("result/EIIE/test_result/action1.npy")

In [60]:
action_list_2 = np.load("result/"+ model_name + "/test_result/action2.npy")
# action_list_2 = check_action_score(action_list_2)
# action_list_2[0][1] += action_list_2[0][0]
# action_list_2[0][0] = 0
# action_list_2[0][3] += 2 * action_list_2[0][2]
# action_list_2[0][2] = action_list_2[0][2] * -1
# np.save("result/test/test_result/action2.npy",action_list_2)
action_list_2


array([[0.05506721, 0.05506252, 0.05504772, 0.05506863, 0.05505133,
        0.05506104, 0.05507012, 0.05505479, 0.05508158, 0.05505873,
        0.05505456, 0.05506245, 0.05508652, 0.05506414, 0.2291086 ],
       [0.05504951, 0.05507445, 0.05507239, 0.05507525, 0.05503763,
        0.05505915, 0.0550769 , 0.05507205, 0.05507666, 0.05504709,
        0.05506409, 0.05506004, 0.0550682 , 0.05505387, 0.22911271],
       [0.05506599, 0.05506153, 0.05506159, 0.05506141, 0.05507484,
        0.05506498, 0.05505364, 0.05505507, 0.05505636, 0.05507589,
        0.05507168, 0.05506003, 0.05506267, 0.05505508, 0.22911912],
       [0.05505262, 0.0550669 , 0.05507002, 0.05506155, 0.0550674 ,
        0.0550649 , 0.05506051, 0.05505985, 0.05506546, 0.05505773,
        0.05506142, 0.05506307, 0.0550624 , 0.05506479, 0.22912142],
       [0.05506465, 0.05506876, 0.05505706, 0.05505644, 0.05506563,
        0.05506507, 0.05505784, 0.05506052, 0.05505832, 0.05506758,
        0.05506288, 0.05505645, 0.05505912, 

In [43]:
action_list_1 = np.load("result/"+ model_name + "/test_result/action1.npy")
# action_list_1 = check_action_score(action_list_1)
action_list_1
# np.save("result/test/test_result/action1.npy",action_list_1)

array([[0.06422535, 0.06422535, 0.06424173, 0.06422389, 0.06422836,
        0.06422964, 0.06423414, 0.06421691, 0.06421633, 0.06423026,
        0.06422725, 0.0642281 , 0.06422529, 0.064224  , 0.10082341],
       [0.06423825, 0.06422286, 0.06422228, 0.06422236, 0.06421922,
        0.06423262, 0.06421771, 0.06422485, 0.06422064, 0.0642316 ,
        0.06423212, 0.06423406, 0.06422292, 0.06423196, 0.10082658],
       [0.06422468, 0.06423131, 0.06422414, 0.06422181, 0.06422811,
        0.06422304, 0.06423397, 0.06423504, 0.06423292, 0.06421617,
        0.06422628, 0.06422451, 0.06422757, 0.06422588, 0.10082458],
       [0.06422511, 0.06422494, 0.06423227, 0.06422247, 0.06423376,
        0.06422402, 0.06422366, 0.06422921, 0.06422397, 0.06422947,
        0.06423009, 0.06422406, 0.06423847, 0.0642229 , 0.10081563],
       [0.06422473, 0.06422266, 0.06422472, 0.06422916, 0.06422483,
        0.06423   , 0.0642262 , 0.06422433, 0.06422804, 0.06422579,
        0.06423083, 0.06422936, 0.06422968, 

In [39]:
for i in action_list_2:
    print(sum(i))
# print(len(action_list_1[7]))
# len(action_list_1[7])
# print(action_list_1)
# sum(action_list_1[7])

1.0000000298023224
1.0000000223517418
0.9999999850988388
0.9999999776482582
0.9999999925494194
0.9999999403953552
0.9999999105930328
1.0000000521540642
0.9999999776482582


In [2]:
np.random.rand(16)

array([0.82330118, 0.80054858, 0.84710509, 0.9555422 , 0.28339452,
       0.66057517, 0.9465914 , 0.5065532 , 0.71509163, 0.19289542,
       0.7381412 , 0.5534191 , 0.1316201 , 0.63032038, 0.00503684,
       0.47246904])

In [12]:
import numpy as np
a = np.empty((5,10),np.float32)
# a[:] = np.array(10) + np.random.randint(-2,2,5).reshape(5,1)
a[:] = 10
a

array([[10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10., 10., 10., 10., 10., 10.]], dtype=float32)

In [16]:
np.random.randint(-2,2,5).reshape(5,1)

array([[-1],
       [ 0],
       [-2],
       [ 0],
       [ 0]])

In [17]:
np.array(range(10)) + np.random.randint(-2,2,5).reshape(5,1)


array([[-1,  0,  1,  2,  3,  4,  5,  6,  7,  8],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
       [-2, -1,  0,  1,  2,  3,  4,  5,  6,  7],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])

In [35]:
rnn = nn.LSTM(10, 20, 2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(2, 3, 20)
c0 = torch.randn(2, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))
output.size()

torch.Size([5, 3, 20])